In [2]:
import numpy as np
import matplotlib.pyplot as plt
import astropy
import pandas as pd

In [ ]:
catalogue['e']

,GALEX_ID,photoextractid,mpstype,avaspra,avaspdec,fexptime,nexptime,GALEX_RA,GALEX_DEC,GALEX_glon,...,usno_blue2,run,rerun,camcol,field,dstArcSec,distanceRank,reverseDistanceRank,multipleMatchCount,reverseMultipleMatchCount
0,6371267606725266443,6371267606724214784,AIS,240.786234,42.421962,189.0,328.0,241.425548,42.425157,67.243057,...,NaN,3225,301,1,204,1.643047,1,1,1,1
1,6371267604577780048,6371267604576731136,AIS,243.425441,42.647581,158.0,366.0,243.439384,42.144146,66.665840,...,21.520000,3225,301,3,212,2.762804,1,2,1,2
2,6371267604577780541,6371267604576731136,AIS,243.425441,42.647581,158.0,366.0,243.218166,42.247637,66.830811,...,NaN,3225,301,3,211,1.201369,1,1,1,2
3,6374539762993202404,6374539762992152576,AIS,196.253760,51.578755,96.0,188.0,196.464140,51.295113,117.453888,...,NaN,3838,301,2,22,1.227040,1,1,1,1
4,6374575020379735318,6374575020378685440,AIS,207.927491,45.346373,112.0,112.0,207.360831,45.144697,94.001389,...,NaN,3716,301,3,143,1.428784,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
110879,6371302790025711030,6371302790022561792,AIS,256.815031,44.668762,142.0,142.0,256.997201,44.854093,70.274773,...,20.650000,5414,301,4,15,0.582907,1,1,1,1
110880,6371302790025711427,6371302790022561792,AIS,256.815031,44.668762,142.0,142.0,256.719890,44.941947,70.369652,...,19.740000,5414,301,4,14,2.951794,1,1,1,1
110881,6371162150247862601,6371162150244712448,AIS,262.261298,44.281834,219.0,225.0,262.653436,44.494382,70.309486,...,21.049999,5414,301,4,42,1.613253,1,1,1,1
110882,6371162150247863017,6371162150244712448,AIS,262.261298,44.281834,219.0,225.0,262.568342,44.572761,70.392456,...,20.289999,5414,301,4,41,2.610218,1,1,2,1


In [131]:
catalogue=pd.read_csv('MyTable_11_gauravsk.csv')


/var/folders/74/__d4fvn923b5nsfrydw7vf4m0000gn/T/ipykernel_51197/263945696.py:1: DtypeWarning: Columns (93,197,198,200) have mixed types. Specify dtype option on import or set low_memory=False.
  catalogue=pd.read_csv('MyTable_11_gauravsk.csv')


In [132]:
_005_LRD_temp = pd.read_csv('/Users/gauravsenthilkumar/repositories/Little-Red-Dots/Base Code/lrd_photometry_005.csv')
_015_LRD_temp = pd.read_csv('/Users/gauravsenthilkumar/repositories/Little-Red-Dots/Base Code/lrd_photometry_015.csv')
_025_LRD_temp = pd.read_csv('/Users/gauravsenthilkumar/repositories/Little-Red-Dots/Base Code/lrd_photometry_025.csv')
_035_LRD_temp = pd.read_csv('/Users/gauravsenthilkumar/repositories/Little-Red-Dots/Base Code/lrd_photometry_035.csv')

In [133]:
def rms_filter_nan_safe(templates, data, thresh=1.0):
    """
    templates: shape (6, 20)
    data: shape (6, 280000)
    Returns: list of boolean arrays, each shape (280000,)
             where True means RMS < thresh, ignoring NaNs
    """
    results = []
    for i in range(len(data)):
        template = templates  # shape (6, 1)
        # Mask valid positions (True where neither is NaN)
        valid_mask = ~np.isnan(template) & ~np.isnan(data[i])  # shape (6, 280000)
        # Differences, ignoring NaNs
        diff = np.where(valid_mask, data[i] - template, 0.0)  # fill NaNs with 0 so they don't affect sum
        # Count valid entries per column
        counts = np.sum(valid_mask)
        # Avoid division by zero
        rms = np.sqrt(np.nansum(diff**2) / counts)
        # Compare to threshold
        mask = (rms < thresh) 
        results.append(mask)
    return results


In [134]:
#dust parameters

R_fuv=8.02
R_nuv=6.72

R_u=4.234
R_g=3.303
R_r=2.285
R_i=1.698
R_z=1.263

In [163]:
def Colour_vector_template(data,with_fuv=False):
    if with_fuv:
        FUV_r= data['FUV']-data['NUV']
        NUV_r= data['NUV']-data['U']
        U_r  = data["U"]-data['G']
        G_r  = data['G']-data['R']
        I_r  = data['R']-data['I']
        Z_r  = data['I']-data['Z']
        NUV_Z = data['NUV']-data['Z']
        return np.array([FUV_r,NUV_r,U_r, G_r,I_r,Z_r,NUV_Z]).T 

    NUV_r= data['NUV']-data['U']
    U_r  = data["U"]-data['G']
    G_r  = data['G']-data['R']
    I_r  = data['R']-data['I']
    Z_r  = data['I']-data['Z']
    NUV_Z = data['NUV']-data['Z']
    return np.array([NUV_r,U_r, G_r,I_r,NUV_Z]).T

def Colour_vector_target(data,with_fuv=False):
    ebv= data['e_bv']
    r=data['psfMag_r']-R_r*ebv
    if with_fuv:
        
        FUV_r= (data['fuv_mag']-R_fuv*ebv) - (data['nuv_mag']-R_nuv*ebv)
        NUV_r= (data['nuv_mag']-R_nuv*ebv) -(data['psfMag_u']- R_u*ebv)
        U_r  = (data["psfMag_u"]-R_u*ebv) - (data['psfMag_g']-R_g*ebv)
        G_r  = (data['psfMag_g']-R_g*ebv) - r
        I_r  = (r) - (data['psfMag_i']-R_i*ebv)
        Z_r  = (data['psfMag_i']-R_i*ebv) - (data['psfMag_z']-R_z*ebv)
        NUV_Z= (data['nuv_mag']-R_fuv*ebv) - (data['psfMag_z']-R_z*ebv)
        return np.array([FUV_r,NUV_r,U_r, G_r,I_r,Z_r,NUV_Z]).T
    NUV_r= (data['nuv_mag']-R_nuv*ebv) -(data['psfMag_u']- R_u*ebv)
    U_r  = (data["psfMag_u"]-R_u*ebv) - (data['psfMag_g']-R_g*ebv)
    G_r  = (data['psfMag_g']-R_g*ebv) - r
    I_r  = (r) - (data['psfMag_i']-R_i*ebv)
    Z_r  = (data['psfMag_i']-R_i*ebv) - (data['psfMag_z']-R_z*ebv)
    NUV_Z = (data['nuv_mag']-R_fuv*ebv) - (data['psfMag_z']-R_z*ebv)
    
    return np.array([NUV_r,U_r, G_r,I_r,Z_r,NUV_Z]).T


In [164]:
LRD_005 = Colour_vector_template(_005_LRD_temp,with_fuv=True)
LRD_015 = Colour_vector_template(_015_LRD_temp,with_fuv=True)
LRD_025 = Colour_vector_template(_025_LRD_temp,with_fuv=True)
LRD_035 = Colour_vector_template(_035_LRD_temp,with_fuv=True)

In [ ]:
len(LRD_005)

20

In [90]:
Target[0]

array([-1.02250465e+03, -4.19261929e-01,  7.16166495e-02, -1.88756278e-01,
        3.29816379e-01, -1.04264444e+00,  1.37246082e+00])

In [165]:
Target= Colour_vector_target(catalogue,with_fuv=True)

In [184]:
from tqdm import tqdm

def RMS_filter_target_fast_tqdm(data, templates, thresh=1.0):
    """
    Compare each row in `data` to all rows in `templates`
    using RMS difference, ignoring NaNs.
    Returns a boolean array (n_data,) where True means
    that row matched at least one template within the threshold.
    """
    data_mask = np.isfinite(data)            # (n_data, cols)
    template_mask = np.isfinite(templates)   # (n_templates, cols)

    results = []
    for j in tqdm(range(data.shape[0]), desc="RMS filtering"):
        
        valid_mask = template_mask & data_mask[j]  # (n_templates, cols)

        diff = np.where(valid_mask, templates - data[j], 0.0)  # (n_templates, cols)

        counts = np.sum(valid_mask, axis=1)  # (n_templates,)
        counts = np.where(counts == 0, np.nan, counts)

        rms = np.sqrt(np.nansum(diff**2, axis=1) / counts)

        results.append(np.any(rms < thresh))

    print('total matches:', np.sum(results))

    return np.array(results, dtype=bool)


True_masks_LRD_005 = RMS_filter_target_fast_tqdm(Target, LRD_005, thresh=0.25)
True_masks_LRD_015 = RMS_filter_target_fast_tqdm(Target, LRD_015, thresh=0.25)
True_masks_LRD_025 = RMS_filter_target_fast_tqdm(Target, LRD_025, thresh=0.25)
True_masks_LRD_035 = RMS_filter_target_fast_tqdm(Target, LRD_035, thresh=0.25)


RMS filtering: 100%|██████████| 110884/110884 [00:06<00:00, 16278.46it/s]


total matches: 185


RMS filtering: 100%|██████████| 110884/110884 [00:06<00:00, 16232.72it/s]


total matches: 1188


RMS filtering: 100%|██████████| 110884/110884 [00:06<00:00, 16662.92it/s]


total matches: 3579


RMS filtering: 100%|██████████| 110884/110884 [00:06<00:00, 16260.50it/s]

total matches: 1779


In [194]:
i=True_masks_LRD_025
mask= (catalogue[i]['psfMag_r']< 19) 
catalogue[i]['SDSS_RA'][mask][:10], catalogue[i]['SDSS_DEC'][mask][:10]

# catalogue[i][mask].to_csv('LRD_005.csv',index=Falsef)

(57711    225.480138
 88223    212.584751
 Name: SDSS_RA, dtype: float64,
 57711    39.095808
 88223    45.270985
 Name: SDSS_DEC, dtype: float64)